# Opex Analytics Case - Data Cleansing, Merging, and Features

The data used to perform these data cleansing activities and feature engineering cannot be provided, but this notebook demonstrates a portion of the activities we are allowed to share.

In [14]:
# imports
import os
import pandas as pd
import numpy as np
from IPython.display import display

In [15]:
# relative path for ease of use between team members
data_path = os.getcwd() + "/Project Week Data.xlsx"

In [16]:
# the data provided from relevant parties in the business problem
tdf_data = pd.read_excel(data_path, sheet_name='Tiera Del Fuego data')
pazzo_data = pd.read_excel(data_path, sheet_name='Pazzo\'s data')

In [17]:
# Our team limited the scope of our analysis to product IDs SKU033, SKU002, and SKU048
scope = ['SKU033', 'SKU002', 'SKU048']

In [18]:
# Filter the data by the scope 
tdf_data = tdf_data.loc[np.isin(tdf_data['SKU'], scope), :]
pazzo_data = pazzo_data.loc[np.isin(pazzo_data['SKU'], scope), :]

# Feature Engineering

### Feature 1 - Absolute Difference between the Purchase Order Units and the Fulfilled Units
- As the business objective is to reduce the difference between the purchase order units and the fulfilled units, the difference between these numerical quantitaties was the metric to gauge performance.

In [19]:
# For each product determine the absolute difference in fulfillment over the total orders for that product
tdf_data['POF Difference'] = np.abs(tdf_data['Purchase Order Units'] - tdf_data['Fulfilled Units'])
tdf_data['POF Difference'] = np.nan_to_num(tdf_data['POF Difference']) # return 0 if there were no orders that week

### Feature 2 - Fulfilled flag to indicate whether an order was a cut or not
- Similar to feature 1, the purpose of this is to create a categorical variable that indicates whether or not the order was fulfilled. This format is preferred to simplify the interpretation of the regression model.

In [20]:
# Fulfilled? (T or F) 
tdf_data['Fulfilled Flag'] = tdf_data['Purchase Order Units'] == tdf_data['Fulfilled Units']

In [21]:
# preview the data
tdf_data.head()

,DC,Date,SKU,Purchase Order Units,Fulfilled Units,POF Difference,Fulfilled Flag
104,DC East,2017-10-02,SKU002,141,141,0,True
105,DC East,2017-10-09,SKU002,1212,0,1212,False
106,DC East,2017-10-16,SKU002,2284,2284,0,True
107,DC East,2017-10-23,SKU002,0,0,0,True
108,DC East,2017-10-30,SKU002,1022,1022,0,True


### Distribution Center level data aggregation
- Another business objective was to understand the difference in demand between the two distribution centers used by the client. Thus, our team aggregated the data by each for ease of exploratory data analysis.

In [22]:
# split by the two distribution centers
tdf_data_east = tdf_data.loc[tdf_data['DC'] == 'DC East', :]
tdf_data_west = tdf_data.loc[tdf_data['DC'] == 'DC West', :]

tdf_data_east = tdf_data_east.rename(columns={"Purchase Order Units": "Purchase Order Units East", "Fulfilled Units" : "Fulfilled Units East", "POF Difference":"POF Difference East", "Fulfilled Flag":"Fulfilled East"})
tdf_data_west = tdf_data_west.rename(columns={"Purchase Order Units": "Purchase Order Units West", "Fulfilled Units" : "Fulfilled Units West", "POF Difference":"POF Difference West", "Fulfilled Flag":"Fulfilled West"})

tdf_data_east = tdf_data_east.drop(columns=['DC'])
tdf_data_west = tdf_data_west.drop(columns=['DC'])

In [23]:
# merge the data again to delineate the attributes of each distribution center
tdf_data = pd.merge(tdf_data_east, tdf_data_west, how='inner', left_on=['Date', 'SKU'], right_on=['Date', 'SKU'])

In [25]:
# calculate additional sub features of feature 1 and feature 2 for each Distribution Center
tdf_data['Both Fulfilled'] = tdf_data.apply(lambda x: x['Fulfilled East'] & x['Fulfilled West'], axis = 1)
tdf_data['Total Purchase Order Units'] = tdf_data.apply(lambda x: x['Purchase Order Units East'] + x['Purchase Order Units West'], axis = 1)
tdf_data['Total Fulfilled Units'] = tdf_data.apply(lambda x: x['Fulfilled Units East'] + x['Fulfilled Units West'], axis = 1)
tdf_data['Total POF Difference'] = tdf_data.apply(lambda x: x['POF Difference East'] + x['POF Difference West'], axis = 1)

In [26]:
tdf_data.head()

,Date,SKU,Purchase Order Units East,Fulfilled Units East,POF Difference East,Fulfilled East,Purchase Order Units West,Fulfilled Units West,POF Difference West,Fulfilled West,Both Fulfilled,Total Purchase Order Units,Total Fulfilled Units,Total POF Difference
0,2017-10-02,SKU002,141,141,0,True,4369,874,3495,False,False,4510,1015,3495
1,2017-10-09,SKU002,1212,0,1212,False,3495,3495,0,True,False,4707,3495,1212
2,2017-10-16,SKU002,2284,2284,0,True,0,0,0,True,True,2284,2284,0
3,2017-10-23,SKU002,0,0,0,True,0,0,0,True,True,0,0,0
4,2017-10-30,SKU002,1022,1022,0,True,1200,0,1200,False,False,2222,1022,1200


### Feature 3 - Average price lookback for a period of 1 - 4 weeks
- Our team hypothesized that price would be a valuable indicator, however the volatility required us to smooth the data through a moving average. Thus, the average price lookback was created to address these issues based on the available price data.

In [27]:
# extract and sort the price data for each of the product types of interest
sku_033 = pazzo_data.loc[pazzo_data['SKU'] == scope[0], :].sort_values(by=['Date']).reset_index(drop=True)
sku_002 = pazzo_data.loc[pazzo_data['SKU'] == scope[1], :].sort_values(by=['Date']).reset_index(drop=True)
sku_048 = pazzo_data.loc[pazzo_data['SKU'] == scope[2], :].sort_values(by=['Date']).reset_index(drop=True)

In [28]:
# for the first 3 entries that don't have data for 4 weeks prior, use the latest week we have data for (week 1)
for i in range(1,5,1):
    sku_033.loc[:,'Average Price '+str(i)+'w'] = np.nan_to_num(sku_033['Average Price'].shift(i), nan=8.395731)
    sku_002.loc[:,'Average Price '+str(i)+'w'] = np.nan_to_num(sku_002['Average Price'].shift(i), nan=13.009440)
    sku_048.loc[:,'Average Price '+str(i)+'w'] = np.nan_to_num(sku_048['Average Price'].shift(i), nan=15.616398)

In [29]:
# combine the data sets once again
pazzo_data = pd.concat([sku_002, sku_033, sku_048]).reset_index(drop=True)

In [30]:
# calculate the moving averages
pazzo_data['1w Average Price'] = pazzo_data.apply(lambda x: np.mean([x['Average Price'], x['Average Price 1w']]), axis = 1)
pazzo_data['2w Average Price'] = pazzo_data.apply(lambda x: np.mean([x['Average Price'], x['Average Price 1w'], x['Average Price 2w']]), axis = 1)
pazzo_data['3w Average Price'] = pazzo_data.apply(lambda x: np.mean([x['Average Price'], x['Average Price 1w'], x['Average Price 2w'], x['Average Price 3w']]), axis = 1)
pazzo_data['4w Average Price'] = pazzo_data.apply(lambda x: np.mean([x['Average Price'], x['Average Price 1w'], x['Average Price 2w'], x['Average Price 3w'], x['Average Price 4w']]), axis = 1)

In [31]:
# create new average price that uses price from past X weeks
pazzo_data = pazzo_data.drop(columns = ['Average Price 1w', 'Average Price 2w', 'Average Price 3w', 'Average Price 4w'])

## Offsetting the time-based data
- Our team realized upon careful inspection of the attributes that data logged within the same week did not actually get become available at the same time during the week. In fact, certain elements needed to be treated as available 1 week past the listed date in the data. Thus, the data needed to be re-sorted and offset based on a subset of attributes.

In [32]:
# re-sort the data by product type and date
sku_033 = pazzo_data.loc[pazzo_data['SKU'] == scope[0], :].sort_values(by=['Date']).reset_index(drop=True)
sku_002 = pazzo_data.loc[pazzo_data['SKU'] == scope[1], :].sort_values(by=['Date']).reset_index(drop=True)
sku_048 = pazzo_data.loc[pazzo_data['SKU'] == scope[2], :].sort_values(by=['Date']).reset_index(drop=True)

In [33]:
# offset the data by 1 week for attributes that are not available in the same week
pazzo_columns_to_shift = ['Consumer Ordered Units', 'Average Price', '1w Average Price', '2w Average Price', '3w Average Price', '4w Average Price', 'Pageview Out of Stock', 'Change in Pageviews']
pazzo_columns_to_keep = ['SKU', 'Date', 'On Hand Inventory', 'Forecast 1 Week', 'Forecast 4 Week', 'Forecast 8 Week']
sku_033 = pd.concat([sku_033[pazzo_columns_to_keep], sku_033[pazzo_columns_to_shift].shift(1)], axis = 1)
sku_002 = pd.concat([sku_002[pazzo_columns_to_keep], sku_002[pazzo_columns_to_shift].shift(1)], axis = 1)
sku_048 = pd.concat([sku_048[pazzo_columns_to_keep], sku_048[pazzo_columns_to_shift].shift(1)], axis = 1)
pazzo_data = pd.concat([sku_002, sku_033, sku_048]).reset_index(drop=True)

## Preview of each data set with features

In [34]:
tdf_data.head(5)

,Date,SKU,Purchase Order Units East,Fulfilled Units East,POF Difference East,Fulfilled East,Purchase Order Units West,Fulfilled Units West,POF Difference West,Fulfilled West,Both Fulfilled,Total Purchase Order Units,Total Fulfilled Units,Total POF Difference
0,2017-10-02,SKU002,141,141,0,True,4369,874,3495,False,False,4510,1015,3495
1,2017-10-09,SKU002,1212,0,1212,False,3495,3495,0,True,False,4707,3495,1212
2,2017-10-16,SKU002,2284,2284,0,True,0,0,0,True,True,2284,2284,0
3,2017-10-23,SKU002,0,0,0,True,0,0,0,True,True,0,0,0
4,2017-10-30,SKU002,1022,1022,0,True,1200,0,1200,False,False,2222,1022,1200


In [35]:
pazzo_data.head()

,SKU,Date,On Hand Inventory,Forecast 1 Week,Forecast 4 Week,Forecast 8 Week,Consumer Ordered Units,Average Price,1w Average Price,2w Average Price,3w Average Price,4w Average Price,Pageview Out of Stock,Change in Pageviews
0,SKU002,2017-10-02,75805,16824,15271,16263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SKU002,2017-10-09,51177,9152,9183,9213,10116.0,13.009440,13.009440,13.009440,13.009440,13.009440,0.000000,0.351913
2,SKU002,2017-10-16,51061,12988,12191,13136,5651.0,13.819208,13.414324,13.279363,13.211882,13.171394,0.000000,0.409566
3,SKU002,2017-10-23,39702,7633,7608,8752,5187.0,13.516317,13.667763,13.448322,13.338601,13.272769,0.000000,0.283794
4,SKU002,2017-10-30,50519,12380,11605,12760,4841.0,12.065530,12.790924,13.133685,13.102624,13.083987,0.005429,0.018394


# Export the Data

In [37]:
# merge both of the datasets for ease of use in one file
combined_data = pd.merge(pazzo_data, tdf_data, how='inner', left_on=['Date', 'SKU'], right_on=['Date', 'SKU'])
pazzo_data.to_csv('pazzo_updated.csv',index=False)
tdf_data.to_csv('tdf_updated.csv',index=False)
combined_data.to_csv('combined_pazzo_tdf.csv', index=False)

In [38]:
combined_data.head()

,SKU,Date,On Hand Inventory,Forecast 1 Week,Forecast 4 Week,Forecast 8 Week,Consumer Ordered Units,Average Price,1w Average Price,2w Average Price,...,POF Difference East,Fulfilled East,Purchase Order Units West,Fulfilled Units West,POF Difference West,Fulfilled West,Both Fulfilled,Total Purchase Order Units,Total Fulfilled Units,Total POF Difference
0,SKU002,2017-10-02,75805,16824,15271,16263,NaN,NaN,NaN,NaN,...,0,True,4369,874,3495,False,False,4510,1015,3495
1,SKU002,2017-10-09,51177,9152,9183,9213,10116.0,13.009440,13.009440,13.009440,...,1212,False,3495,3495,0,True,False,4707,3495,1212
2,SKU002,2017-10-16,51061,12988,12191,13136,5651.0,13.819208,13.414324,13.279363,...,0,True,0,0,0,True,True,2284,2284,0
3,SKU002,2017-10-23,39702,7633,7608,8752,5187.0,13.516317,13.667763,13.448322,...,0,True,0,0,0,True,True,0,0,0
4,SKU002,2017-10-30,50519,12380,11605,12760,4841.0,12.065530,12.790924,13.133685,...,0,True,1200,0,1200,False,False,2222,1022,1200
